In [1]:

import pandas as pd
import json
import pathlib
import sys
import concurrent.futures
import os
import numpy as np


In [2]:

sys.path.insert(0, "..\\controller")
from tile_configuration import TileConfiguration


In [3]:

prefix = pathlib.Path(
    "C:\\Users\\davidek\\microscope_ui\\controller\\photo\\1732508547.7836869"
)

tc = TileConfiguration()
tc.load(prefix / "TileConfiguration.registered.txt")
tc.move_to_origin()
tc.images = tc.images[:10]



In [4]:

def load(filename):
    with open(filename) as r:
        return json.load(r)


In [13]:

d = []

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    future_to_fname = {
        executor.submit(
            load,
            pathlib.Path("..\\out")
            / image.filename.with_suffix(image.filename.suffix + ".json"),
        ): image.filename
        for image in tc.images
    }

    for future in concurrent.futures.as_completed(future_to_fname):
        filename = future_to_fname[future]
        results = future.result()
        for result in results:
            d.append( (filename, str(pathlib.Path(result)), results[result][0], results[result][1]))


In [14]:
p = pd.DataFrame(d, columns=['fname1', 'fname2', 'tx', 'ty'])
p

,fname1,fname2,tx,ty
0,test.1732508557.2022848.png,test.1732508594.1306095.png,182.0,320.0
1,test.1732508557.2022848.png,test.1732508592.2495918.png,0.0,458.0
2,test.1732508557.2022848.png,test.1732508639.7550175.png,182.0,388.0
3,test.1732508557.2022848.png,test.1732508637.907108.png,0.0,478.0
4,test.1732508557.2022848.png,test.1732508685.3354325.png,174.0,388.0
...,...,...,...,...
6624,test.1732508560.9531374.png,test.1732512800.1110008.png,214.0,266.0
6625,test.1732508560.9531374.png,test.1732512818.5151327.png,212.0,302.0
6626,test.1732508560.9531374.png,test.1732512816.6818404.png,220.0,276.0
6627,test.1732508560.9531374.png,test.1732512814.854225.png,214.0,276.0


In [ ]:
p['mag'] = np.sqrt(p['tx'] ** 2 + p['ty'] ** 2)

In [ ]:
#p.sort_values(by=['mag'], inplace=True)
#p.reset_index()
#p['index_col'] = p.index

In [ ]:
p.to_csv("test.csv")

In [ ]:
p.to_json("test.json", orient="records")

OverflowError: Maximum recursion level reached

In [ ]:
def combine_two_images(i1, i2, tx, ty):
    # find the boundary of the union
    # paste the two images into the union
